# 基本概念

“协程”（Coroutine）概念最早由 Melvin Conway 于1958年提出。协程可以理解为纯用户态的线程，其通过协作而不是抢占来进行切换。相对于进程或者线程，协程所有的操作都可以在用户态完成，创建和切换的消耗更低。总的来说，协程为协同任务提供了一种运行时抽象，这种抽象非常适合于协同多任务调度和数据流处理。在现代操作系统和编程语言中，因为用户态线程切换代价比内核态线程小，协程成为了一种轻量级的多任务模型。

从编程角度上看，协程的思想本质上就是控制流的主动让出（yield）和恢复（resume）机制，迭代器常被用来实现协程，所以大部分的语言实现的协程中都有yield关键字，比如Python、PHP、Lua。但也有特殊比如Go就使用的是通道来通信。

# 协程与进程线程的区别

- 对于操作系统来说只有进程和线程，协程的控制由应用程序显式调度，非抢占式的
- 协程的执行最终靠的还是线程，应用程序来调度协程选择合适的线程来获取执行权
- 切换非常快，成本低。一般占用栈大小远小于线程（协程KB级别，线程MB级别），所以可以开更多的协程
- 协程比线程更轻量级

In [1]:
import time, random

"""从 yield 引出的, 本身不会直接将全部结果计算出来，而是会生成一个迭代器，"""
def fib(n):
    index = 0
    a = 0
    b = 1
    while index < n:
        yield b
        a, b = b, a + b
        index += 1


for i in fib(20):
    pass # 1, 1, 2, 3, 5, 8...

In [10]:
""" 可以通过send 函数，与yield 流出配合，完成通信 """
# 在此例子中，通过send(4) 可赋予函数中 sleep_cnt 值
def stupid_fib(n):
    index = 0
    a = 0
    b = 1
    while index < n:
        sleep_cnt = yield b
        print("recv" + str(sleep_cnt))
        a, b = b, a + b
        index += 1
N = 20
sfib = stupid_fib(N)
fib_res = next(sfib)
print(fib_res)
sfib.send(4)

1
recv4


1

In [13]:
"""yield from 用于重构生成器， 还可以像管道一样将send信息传递给内层协程"""
def copy_fib(n):
    print('I am copy from fib')
    yield from fib(n)
    print('Copy end')
for fib_res in copy_fib(4):
    print(fib_res)

I am copy from fib
1
1
2
3
Copy end


# 协程 asyncio (async/await)

```

import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

# Python 3.7+
asyncio.run(main())

```

In [14]:
import asyncio

In [21]:
async def compute(x, y):
    print('Compute ' + str(x) + '+' + str(y))
    await asyncio.sleep(1)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print(str(x) + '+' + str(y) + '=' + str(result))
    
loop = asyncio.get_event_loop()
if not loop.is_running:
    loop.run_until_complete(print_sum(1, 2)) # 
    loop.close()